In [202]:
from gephistreamer import 

In [237]:
h = gephistreamer.

In [ ]:
h.

In [204]:
nx.write_gexf(G, "test.gexf")

In [220]:
supabase.storage().from_("test2").upload(path="./", file="test.gexf")

StorageException: {'statusCode': 400, 'error': 'Invalid JWT', 'message': 'new row violates row-level security policy for table "objects"'}

In [227]:
s = supabase.storage()

In [228]:
s.get_bucket('postthread-bucket')

StorageException: {'statusCode': 400, 'error': 'Not found', 'message': 'The resource was not found'}

In [216]:
ss = s.from_("postthread-bucket")

In [217]:
ss.upload(path="./", file="test.gexf")

StorageException: {'statusCode': 400, 'error': 'Invalid JWT', 'message': 'new row violates row-level security policy for table "objects"'}

In [1]:
%load_ext autoreload
%autoreload 2

In [126]:
from scripts.web3_helpers import *
import hmac
import hashlib
import base64
from web3 import Web3
import os, json
from supabase import create_client, Client
import pandas as pd
import math
import praw
import networkx as nx

In [10]:
creds = json.load(open(".db-creds.json"))

url = creds["SUPABASE_URL"]
key = creds["SUPABASE_KEY"]
pw = creds["PW_HASH"]
supabase = create_client(url, key)

In [100]:
reddit_creds = json.load(open(".reddit_creds.json", "r"))
reddit = praw.Reddit(
    client_id=reddit_creds["client_id"],
    client_secret=reddit_creds["client_secret"],
    password=reddit_creds["password"],
    user_agent=reddit_creds["user_agent"],
    username=reddit_creds["username"],
)

In [148]:
daily_token_rewards = 100000
rewards = {"posts": 100, "comments": 10, "votes": 1, "follows": 5, "links": 400}

In [136]:
def exp_for_level():
    lvl = 1
    exp = 0
    exp_4_next_lvl = 83
    result = {lvl: 0}
    while lvl < 99:
        exp += exp_4_next_lvl
        exp_4_next_lvl += exp_4_next_lvl * 0.10407
        lvl += 1
        result[lvl] = exp
    return result

lvl_to_exp = exp_for_level()

In [45]:
dig = hmac.new(b'password', msg="PocketAAcez".encode(), digestmod=hashlib.sha256).digest()
base64.b64encode(dig).decode()

'7DR6F/yY56zt9XVYrB4ogIYAo76hlPgL5udpXxgAgr8='

In [119]:
categories = supabase.table("categories").select("*").is_("transaction_hash", "null").execute().data
comments = supabase.table("comments").select("*").not_.is_("ipfs_hash", "null").is_("transaction_hash", "null").execute().data
follows = supabase.table("follows").select("*").is_("transaction_hash", "null").execute().data
# payouts = supabase.table("payouts").select("*").is_("transaction_hash", "null").execute().data
posts = supabase.table("posts").select("*").not_.is_("ipfs_hash", "null").is_("transaction_hash", "null").execute().data
all_users = supabase.table("users").select("*").execute().data
# users = supabase.table("users").select("*").is_("transaction_hash", "null").execute().data
# daily_payouts = supabase.table("users").select("*").eq("daily_payout_claimed", True).execute().data
# airdrop_payouts = supabase.table("users").select("*").eq("reddit_airdrop_claimed", "pending").execute().data
votes = supabase.table("votes").select("*").is_("transaction_hash", "null").execute().data

In [198]:
all_data = {
    "categories": pd.DataFrame(supabase.table("categories").select("*").execute().data),
    "comments": pd.DataFrame(supabase.table("comments").select("*").execute().data),
    "follows": pd.DataFrame(supabase.table("follows").select("*").execute().data),
    "posts": pd.DataFrame(supabase.table("posts").select("*").execute().data),
    "users": pd.DataFrame(supabase.table("users").select("*").execute().data),
    "votes": pd.DataFrame(supabase.table("votes").select("*").execute().data),
}

In [ ]:
# all_data = all_users.join()

In [120]:
# all_users = pd.DataFrame(all_users)

In [140]:
all_data['users'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 771 entries, 0 to 770
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   id                       771 non-null    int64 
 1   created_at               771 non-null    object
 2   username                 1 non-null      object
 3   profile_pic              771 non-null    object
 4   msa_id                   0 non-null      object
 5   transaction_hash         0 non-null      object
 6   wallet_address_personal  0 non-null      object
 7   wallet_address_provided  0 non-null      object
 8   exp                      771 non-null    int64 
 9   level                    771 non-null    int64 
 10  reddit_airdrop_value     0 non-null      object
 11  reddit_airdrop_claimed   771 non-null    object
 12  reddit_username          770 non-null    object
 13  github_username          0 non-null      object
 14  discord_username         0 non-null      o

In [199]:
def get_centralities(follows_data):    
    G = nx.Graph()
    G.add_edges_from(pd.DataFrame(follows_data)[['follower_id', 'following_id']].values.tolist())
    degree_scores = nx.degree_centrality(G)
    closeness_scores = nx.closeness_centrality(G)
    betweeness_scores = nx.betweenness_centrality(G, k=1)
    
    return [degree_scores, closeness_scores, betweeness_scores], G

def get_user_weighted_social_score(centralities, user_id):
    user_centralities = []
    for centrality in centralities:
        if user_id not in centrality.keys():
            user_centralities.append(0)
        else:
            centrality_max = max(centrality.values())
            user_centralities.append(0 if centrality_max == 0 else centrality[user_id] / centrality_max)
    
    user_centralities.sort(reverse=True)
    weighted_avg = 0
    for i, score in enumerate(user_centralities):
        weighted_avg += score * i
        
    return weighted_avg / 6

def get_user_exp(user_id):
    posts = all_data['posts'][all_data['posts']['user_id'] == user_id]
    posts_upvotes = all_data['votes'][all_data['votes']['post_id'].isin(posts['id'])]['up'].sum()
    
    exp = 0
    exp += posts.size * rewards['posts']
    exp += all_data['comments'][all_data['comments']['user_id'] == user_id].size * rewards['comments']
    exp += all_data['votes'][all_data['votes']['user_id'] == user_id].size * rewards['votes']
    exp += posts['reddit_upvotes'].sum() * rewards['votes']
    exp += all_data['follows'][all_data['follows']['follower_id'] == user_id].size * rewards['follows']
    exp += all_data['follows'][all_data['follows']['following_id'] == user_id].size * rewards['follows'] * 2
    
    user = all_data['users'][all_data['users']['id'] == user_id].iloc[0]
    if user['reddit_username'] is not None:
        exp += rewards['links']
    if user['github_username'] is not None:
        exp += rewards['links']
    if user['discord_username'] is not None:
        exp += rewards['links']
    if user['email'] is not None:
        exp += rewards['links']
        
    return exp

def update_scores(df):
    df['id']
    centralities, G = get_centralities(all_data['follows'][all_data['follows']['follower_id'].isin(df['id']) | all_data['follows']['following_id'].isin(df['id'])])
    
    social_score = []
    level = []
    overall_score = []
    exp = []
    exp_to_next_level = []
    
    for index, row in df.iterrows():
        user_id = row['id']
        
        user_exp = get_user_exp(user_id)
        user_level = row['level']
        while lvl_to_exp[user_level] < user_exp:
            user_level += 1
        
        ss = get_user_weighted_social_score(centralities, user_id) * 1000
        social_score.append(max(ss, 1))
        level.append(user_level)
        overall_score.append(user_level * social_score[-1])
        exp.append(user_exp)
        exp_to_next_level.append(lvl_to_exp[user_level + 1] - lvl_to_exp[user_level])
    
    df['social_score'] = social_score
    df['level'] = level
    df['overall_score'] = overall_score
    df['overall_score'] /= max(overall_score)
    df['exp'] = exp
    df['exp_to_next_level'] = exp_to_next_level
    df['daily_payout_value'] = daily_token_rewards * df['overall_score']
    
    return df

In [131]:
centralities, G = get_centralities(follows)

In [200]:
# Daily payout
for idx, row in all_data['users'][all_data['users']['daily_payout_claimed'] == True].iterrows():
    updates = {
        "daily_payout_claimed": False, 
        "tokens_to_claim": row['tokens_to_claim'] + row['daily_payout_value']
    }
    print(updates)
    supabase.table("users").update(updates).eq("id", row['id']).execute()
    
all_users = all_data['users'].copy()
all_users[all_users['username'].isna()] = update_scores(all_users[all_users['username'].isna()])
all_users[~all_users['username'].isna()] = update_scores(all_users[~all_users['username'].isna()])

for idx, row in all_users.iterrows():
    updates = {
        'social_score': row['social_score'],
        'level': row['level'],
#         'overall_score': row['overall_score'],
        'exp': row['exp'],
        'exp_to_next_level': row['exp_to_next_level'],
        'daily_payout_value': row['daily_payout_value'],
    }
    print(updates)
    supabase.table("users").update(updates).eq("id", row['id']).execute()

{'daily_payout_claimed': False, 'tokens_to_claim': 22266.7483705171}


C:\Users\Mcian\AppData\Local\Temp\ipykernel_24452\1917381047.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['social_score'] = social_score
C:\Users\Mcian\AppData\Local\Temp\ipykernel_24452\1917381047.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['level'] = level
C:\Users\Mcian\AppData\Local\Temp\ipykernel_24452\1917381047.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

{'social_score': 1.0, 'level': 6, 'exp': 510, 'exp_to_next_level': 136.16363568137672, 'daily_payout_value': 87.16450174106967}
{'social_score': 1.0, 'level': 6, 'exp': 510, 'exp_to_next_level': 136.16363568137672, 'daily_payout_value': 87.16450174106967}
{'social_score': 1.0, 'level': 6, 'exp': 510, 'exp_to_next_level': 136.16363568137672, 'daily_payout_value': 87.16450174106967}
{'social_score': 1.0, 'level': 6, 'exp': 510, 'exp_to_next_level': 136.16363568137672, 'daily_payout_value': 87.16450174106967}
{'social_score': 1.0, 'level': 6, 'exp': 510, 'exp_to_next_level': 136.16363568137672, 'daily_payout_value': 87.16450174106967}
{'social_score': 1.0, 'level': 6, 'exp': 510, 'exp_to_next_level': 136.16363568137672, 'daily_payout_value': 87.16450174106967}
{'social_score': 1.0, 'level': 6, 'exp': 510, 'exp_to_next_level': 136.16363568137672, 'daily_payout_value': 87.16450174106967}
{'social_score': 1.0, 'level': 6, 'exp': 510, 'exp_to_next_level': 136.16363568137672, 'daily_payout_val

{'social_score': 1.0, 'level': 6, 'exp': 510, 'exp_to_next_level': 136.16363568137672, 'daily_payout_value': 87.16450174106967}
{'social_score': 1.0, 'level': 6, 'exp': 510, 'exp_to_next_level': 136.16363568137672, 'daily_payout_value': 87.16450174106967}
{'social_score': 1.0, 'level': 6, 'exp': 510, 'exp_to_next_level': 136.16363568137672, 'daily_payout_value': 87.16450174106967}
{'social_score': 1.0, 'level': 6, 'exp': 510, 'exp_to_next_level': 136.16363568137672, 'daily_payout_value': 87.16450174106967}
{'social_score': 1.0, 'level': 6, 'exp': 510, 'exp_to_next_level': 136.16363568137672, 'daily_payout_value': 87.16450174106967}
{'social_score': 1.0, 'level': 6, 'exp': 510, 'exp_to_next_level': 136.16363568137672, 'daily_payout_value': 87.16450174106967}
{'social_score': 1.0, 'level': 6, 'exp': 510, 'exp_to_next_level': 136.16363568137672, 'daily_payout_value': 87.16450174106967}
{'social_score': 1.0, 'level': 6, 'exp': 510, 'exp_to_next_level': 136.16363568137672, 'daily_payout_val

{'social_score': 1.0, 'level': 6, 'exp': 510, 'exp_to_next_level': 136.16363568137672, 'daily_payout_value': 87.16450174106967}
{'social_score': 1.0, 'level': 6, 'exp': 510, 'exp_to_next_level': 136.16363568137672, 'daily_payout_value': 87.16450174106967}
{'social_score': 1.0, 'level': 6, 'exp': 510, 'exp_to_next_level': 136.16363568137672, 'daily_payout_value': 87.16450174106967}
{'social_score': 1.0, 'level': 6, 'exp': 510, 'exp_to_next_level': 136.16363568137672, 'daily_payout_value': 87.16450174106967}
{'social_score': 1.0, 'level': 6, 'exp': 510, 'exp_to_next_level': 136.16363568137672, 'daily_payout_value': 87.16450174106967}
{'social_score': 1.0, 'level': 6, 'exp': 510, 'exp_to_next_level': 136.16363568137672, 'daily_payout_value': 87.16450174106967}
{'social_score': 1.0, 'level': 6, 'exp': 510, 'exp_to_next_level': 136.16363568137672, 'daily_payout_value': 87.16450174106967}
{'social_score': 1.0, 'level': 7, 'exp': 620, 'exp_to_next_level': 150.33418524673766, 'daily_payout_val

...

{'social_score': 1.0, 'level': 6, 'exp': 510, 'exp_to_next_level': 136.16363568137672, 'daily_payout_value': 87.16450174106967}
{'social_score': 1.0, 'level': 6, 'exp': 510, 'exp_to_next_level': 136.16363568137672, 'daily_payout_value': 87.16450174106967}
{'social_score': 1.0, 'level': 6, 'exp': 510, 'exp_to_next_level': 136.16363568137672, 'daily_payout_value': 87.16450174106967}
{'social_score': 1.0, 'level': 6, 'exp': 510, 'exp_to_next_level': 136.16363568137672, 'daily_payout_value': 87.16450174106967}
{'social_score': 1.0, 'level': 7, 'exp': 620, 'exp_to_next_level': 150.33418524673766, 'daily_payout_value': 101.6919186979146}
{'social_score': 1.0, 'level': 6, 'exp': 510, 'exp_to_next_level': 136.16363568137672, 'daily_payout_value': 87.16450174106967}
{'social_score': 1.0, 'level': 13, 'exp': 1785, 'exp_to_next_level': 272.2935813710892, 'daily_payout_value': 188.8564204389843}
{'social_score': 1.0, 'level': 6, 'exp': 510, 'exp_to_next_level': 136.16363568137672, 'daily_payout_va

{'social_score': 7.246376811594203, 'level': 7, 'exp': 535, 'exp_to_next_level': 150.33418524673766, 'daily_payout_value': 736.8979615790915}
{'social_score': 14.492753623188406, 'level': 7, 'exp': 560, 'exp_to_next_level': 150.33418524673766, 'daily_payout_value': 1473.795923158183}
{'social_score': 18.605555115763064, 'level': 7, 'exp': 610, 'exp_to_next_level': 150.33418524673766, 'daily_payout_value': 1892.0345981617472}
{'social_score': 14.492753623188406, 'level': 7, 'exp': 585, 'exp_to_next_level': 150.33418524673766, 'daily_payout_value': 1473.795923158183}
{'social_score': 1.0, 'level': 7, 'exp': 620, 'exp_to_next_level': 150.33418524673766, 'daily_payout_value': 101.6919186979146}
{'social_score': 21.73913043478261, 'level': 7, 'exp': 610, 'exp_to_next_level': 150.33418524673766, 'daily_payout_value': 2210.693884737274}
{'social_score': 23.607086410447895, 'level': 7, 'exp': 585, 'exp_to_next_level': 150.33418524673766, 'daily_payout_value': 2400.649911945913}
{'social_score'

{'social_score': 7.246376811594203, 'level': 7, 'exp': 535, 'exp_to_next_level': 150.33418524673766, 'daily_payout_value': 736.8979615790915}
{'social_score': 14.492753623188406, 'level': 7, 'exp': 585, 'exp_to_next_level': 150.33418524673766, 'daily_payout_value': 1473.795923158183}
{'social_score': 1.0, 'level': 6, 'exp': 510, 'exp_to_next_level': 136.16363568137672, 'daily_payout_value': 87.16450174106967}
{'social_score': 7.246376811594203, 'level': 7, 'exp': 560, 'exp_to_next_level': 150.33418524673766, 'daily_payout_value': 736.8979615790915}
{'social_score': 1.0, 'level': 6, 'exp': 510, 'exp_to_next_level': 136.16363568137672, 'daily_payout_value': 87.16450174106967}
{'social_score': 1.0, 'level': 6, 'exp': 510, 'exp_to_next_level': 136.16363568137672, 'daily_payout_value': 87.16450174106967}
{'social_score': 7.246376811594203, 'level': 7, 'exp': 535, 'exp_to_next_level': 150.33418524673766, 'daily_payout_value': 736.8979615790915}
{'social_score': 1.0, 'level': 6, 'exp': 510, '

{'social_score': 1.0, 'level': 13, 'exp': 1803, 'exp_to_next_level': 272.2935813710892, 'daily_payout_value': 188.8564204389843}
{'social_score': 1.0, 'level': 6, 'exp': 510, 'exp_to_next_level': 136.16363568137672, 'daily_payout_value': 87.16450174106967}
{'social_score': 1.0, 'level': 6, 'exp': 510, 'exp_to_next_level': 136.16363568137672, 'daily_payout_value': 87.16450174106967}
{'social_score': 1.0, 'level': 6, 'exp': 510, 'exp_to_next_level': 136.16363568137672, 'daily_payout_value': 87.16450174106967}
{'social_score': 1.0, 'level': 6, 'exp': 510, 'exp_to_next_level': 136.16363568137672, 'daily_payout_value': 87.16450174106967}
{'social_score': 1.0, 'level': 6, 'exp': 510, 'exp_to_next_level': 136.16363568137672, 'daily_payout_value': 87.16450174106967}
{'social_score': 1.0, 'level': 6, 'exp': 510, 'exp_to_next_level': 136.16363568137672, 'daily_payout_value': 87.16450174106967}
{'social_score': 1.0, 'level': 6, 'exp': 510, 'exp_to_next_level': 136.16363568137672, 'daily_payout_va

{'social_score': 1.0, 'level': 14, 'exp': 1840, 'exp_to_next_level': 300.63117438437894, 'daily_payout_value': 203.3838373958292}
{'social_score': 1.0, 'level': 7, 'exp': 620, 'exp_to_next_level': 150.33418524673766, 'daily_payout_value': 101.6919186979146}
{'social_score': 1.0, 'level': 6, 'exp': 510, 'exp_to_next_level': 136.16363568137672, 'daily_payout_value': 87.16450174106967}
{'social_score': 1.0, 'level': 6, 'exp': 510, 'exp_to_next_level': 136.16363568137672, 'daily_payout_value': 87.16450174106967}
{'social_score': 1.0, 'level': 6, 'exp': 510, 'exp_to_next_level': 136.16363568137672, 'daily_payout_value': 87.16450174106967}
{'social_score': 1.0, 'level': 6, 'exp': 510, 'exp_to_next_level': 136.16363568137672, 'daily_payout_value': 87.16450174106967}
{'social_score': 1.0, 'level': 6, 'exp': 510, 'exp_to_next_level': 136.16363568137672, 'daily_payout_value': 87.16450174106967}
{'social_score': 1.0, 'level': 6, 'exp': 510, 'exp_to_next_level': 136.16363568137672, 'daily_payout_v

In [39]:
# Reddit Airdrop

# Update airdrop value
for idx, row in all_users[!all_users['reddit_username'].isna() && all_users['reddit_airdrop_value'].isna()]:
    user = reddit.redditor(reddit_username)    
    airdrop_value = user.total_karma
    supabase.table("users").update({"reddit_airdrop_value": airdrop_value}).eq("id", row['id']).execute()
                
# Check for claims
for idx, row in all_users[all_users['reddit_airdrop_claimed'] == "pending"]:
    if row['reddit_id'] is None:
        supabase.table("users").update({"reddit_airdrop_claimed": "rejected"}).eq("id", row['id']).execute()
    
    user = reddit.redditor(reddit_username)  
    try:
        for i, post in enumerate(user.new()):
            if i > 10:
                break
            if type(post) == praw.models.reddit.submission.Submission:
                if row["wallet_address_personal"] in post.selftext:
                    airdrop_value = user.total_karma
                    updates = {
                        "reddit_airdrop_value": airdrop_value,
                        "reddit_airdrop_claimed": "claimed",
                        "tokens_to_claim": row['tokens_to_claim'] + airdrop_value,
                    }
                    supabase.table("users").update(updates).eq("id", row['id']).execute()
    except:
        supabase.table("users").update({"reddit_airdrop_claimed": "rejected"}).eq("id", row['id']).execute()

[]